In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu,True)

2024-11-02 08:23:33.029448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 08:23:33.094756: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 08:23:33.114772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 08:23:33.262919: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1730535820.003255    1277 cuda_executor.c

In [3]:
import polars as pl
import numpy as np
import os
directory = '/app/finalDataSets'
# directory = r'D:\github\localt20'

balltoball = tf.convert_to_tensor(pl.read_csv(os.path.join(directory,'balltoball.csv')))

# Shapes of the data

balltoball_shape = balltoball.shape
balltoball_shape

TensorShape([267455, 6])

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define RNN model
input_shape = (None, balltoball_shape[1])  # None allows variable sequence length for live data
rnn_model = models.Sequential([
    layers.Masking(mask_value=0, input_shape=input_shape), # Masking for variable-length data
    layers.LSTM(64, return_sequences=True),  
    layers.LSTM(32),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')   # Output for binary classification
])

rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, None, 6)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 64)       │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,681 (123.75 KB)

 Trainable params: 31,681 (123.75 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Assuming balltoball is of shape (sequence_length, features) = (32, 6)
curr_balls = 150

balltoball_reshaped = tf.reshape(balltoball[:curr_balls], (1, curr_balls, 6))  # Adding a batch dimension

# Now use the reshaped data for prediction
prediction = rnn_model.predict(balltoball_reshaped)
print("Prediction:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: [[0.41954237]]
